In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [3]:
sq_eu = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/maps/QGIS/square_europe_updated.csv')
storm_dates = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/storms_dates_with_steps_and_index.csv')
name_storm = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/C3S_StormTracks_era5_19792021_0100_v3.csv',
                         delimiter=';')

landfall_eu = sq_eu.drop_duplicates(subset=['layer'], keep='first')
landfall_eu = landfall_eu.drop(columns=['lon_east', 'lon_west', 'lat_north', 'lat_south', 'path','center_lon','center_lat'])

name_storm = name_storm[name_storm['Time&Longitude&Latitude'].str.startswith('TRACK_ID')]


In [4]:
# in the column 'layer' rename each fid_reproj_tc_irad_i_interp by storm_i

landfall_eu = landfall_eu.rename(columns={'layer': 'storm_index'})
landfall_eu['storm_index'] = landfall_eu['storm_index'].str.replace('fid_reproj_tc_irad_', '')
landfall_eu['storm_index'] = landfall_eu['storm_index'].str.replace('_interp', '')

# clean the storm_dates dataframe

storm_eu_dates = storm_dates.drop(columns=['end_date', '3_hour_steps', '1_hour_steps', 'Unnamed: 0'])
landfall_eu['storm_index'] = pd.to_numeric(landfall_eu['storm_index'], errors='coerce')

# keep only the name of the storms

name_storm = name_storm['Time&Longitude&Latitude'].str.split(' 0 ', n=1).str[1].str.strip()
name_storm = name_storm.apply(lambda x: x.replace('NAME', '', 1).strip() if x.startswith('NAME') else x)
name_storm.reset_index(drop=True, inplace=True)
name_storm = pd.DataFrame(name_storm)

In [4]:
'''test = storm_eu_dates['start_date'][0+1]
hours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == 1]
hours_to_add = int(hours_to_add['fid'].values[0])
date_time_obj = datetime.strptime(test, '%Y-%m-%dT%H:%M:%S')
date_time_obj = date_time_obj + timedelta(hours=hours_to_add)

# convert back to string

date_time_obj.strftime('%Y-%m-%dT%H:%M:%S')'''

"test = storm_eu_dates['start_date'][0+1]\nhours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == 1]\nhours_to_add = int(hours_to_add['fid'].values[0])\ndate_time_obj = datetime.strptime(test, '%Y-%m-%dT%H:%M:%S')\ndate_time_obj = date_time_obj + timedelta(hours=hours_to_add)\n\n# convert back to string\n\ndate_time_obj.strftime('%Y-%m-%dT%H:%M:%S')"

In [5]:
storm_eu_arrivals = []
steps_added = []

for i in range(len(storm_eu_dates)):
    try:
        start = storm_eu_dates['start_date'][i]
        hours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == i+1]
        hours_to_add = int(hours_to_add['fid'].values[0])
        start_reformat = datetime.strptime(start, '%Y-%m-%dT%H:%M:%S')
        start_reformat = start_reformat + timedelta(hours=hours_to_add)
        storm_eu_arrivals.append(start_reformat.strftime('%Y-%m-%dT%H:%M:%S'))
        steps_added.append(hours_to_add)
    except:
        print(f'storm {i} not found')
        storm_eu_arrivals.append(-1)
        steps_added.append(-1)

storm 90 not found


In [6]:
# add the column 'arrival_date' to storm_dates

storm_dates['eu_landfall_date'] = storm_eu_arrivals
storm_dates['nb_steps_before_eu_landfall_1_hour'] = steps_added
storm_dates['storm_name'] = name_storm

In [7]:
# save the new storm_dates

storm_dates.to_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/storms_dates_with_steps_and_index.csv')